In [2]:
import os
print("SPARK_CONF_DIR =", os.environ.get("SPARK_CONF_DIR"))
print("SPARK_HOME =", os.environ.get("SPARK_HOME"))

SPARK_CONF_DIR = /opt/spark/conf
SPARK_HOME = /opt/spark


In [3]:
conf_path = "/opt/spark/conf/spark-defaults.conf"
with open(conf_path, 'r') as f:
    lines = f.readlines()
print("First 10 lines of spark-defaults.conf:", lines[:10])

First 10 lines of spark-defaults.conf: ['# ==============================================================================\n', '# SPARK CONFIGURATION FOR POLARIS + ICEBERG + MINIO\n', '# ==============================================================================\n', '\n', '# Basic Extensions\n', '# https://iceberg.apache.org/docs/latest/spark-configuration\n', 'spark.sql.extensions                   org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions\n', '\n', '# Catalog Configuration (The Polaris Connection)\n', '# https://iceberg.apache.org/docs/latest/spark-configuration\n']


In [4]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Polaris Debug")
    .getOrCreate()
)

[02:19:43] [Thread-3] WARN  org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Test
print(spark.conf.get("spark.sql.catalog.polaris.uri"))

http://polaris:8181/api/catalog


In [21]:
spark.sql("CREATE NAMESPACE IF NOT EXISTS polaris.olist_raw")

DataFrame[]

In [22]:
spark.sql("SHOW NAMESPACES IN polaris").show()

+---------+
|namespace|
+---------+
|olist_raw|
+---------+



In [23]:
spark.sql("""
    CREATE TABLE IF NOT EXISTS polaris.olist_raw.test_connection (
        id INT,
        status STRING
    ) USING iceberg
""")

[01:43:08] [Thread-3] INFO  org.apache.iceberg.rest.RESTSessionCatalog - Table properties set at catalog level through catalog properties: {}
[01:43:08] [Thread-3] INFO  org.apache.iceberg.rest.RESTSessionCatalog - Table properties enforced at catalog level through catalog properties: {}
[01:43:08] [Thread-3] INFO  org.apache.iceberg.CatalogUtil - Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO


DataFrame[]

In [24]:
import socket
socket.gethostbyname("minio")  # should return the internal IP

'172.18.0.3'

In [25]:
# Insert data
spark.sql("""
INSERT INTO polaris.olist_raw.test_connection VALUES
(1, 'pending'),
(2, 'complete')
""")

# Read back
spark.sql("SELECT * FROM polaris.olist_raw.test_connection").show()

[01:43:12] [Thread-3] INFO  org.apache.iceberg.spark.source.SparkWrite - Committing append with 2 new data files to table polaris.olist_raw.test_connection
[01:43:12] [Thread-3] INFO  org.apache.iceberg.SnapshotProducer - Committed snapshot 8942229491039698229 (MergeAppend)
[01:43:12] [Thread-3] INFO  org.apache.iceberg.metrics.LoggingMetricsReporter - Received metrics report: CommitReport{tableName=polaris.olist_raw.test_connection, snapshotId=8942229491039698229, sequenceNumber=1, operation=append, commitMetrics=CommitMetricsResult{totalDuration=TimerResult{timeUnit=NANOSECONDS, totalDuration=PT0.168677222S, count=1}, attempts=CounterResult{unit=COUNT, value=1}, addedDataFiles=CounterResult{unit=COUNT, value=2}, removedDataFiles=null, totalDataFiles=CounterResult{unit=COUNT, value=2}, addedDeleteFiles=null, addedEqualityDeleteFiles=null, addedPositionalDeleteFiles=null, addedDVs=null, removedDeleteFiles=null, removedEqualityDeleteFiles=null, removedPositionalDeleteFiles=null, removed

In [26]:
# Check what Spark actually loaded
conf = spark.sparkContext.getConf().getAll()
for item in conf:
    if "s3.endpoint" in item[0]:
        print(f"✅ FOUND: {item}")

✅ FOUND: ('spark.sql.catalog.polaris.s3.endpoint', 'http://minio:9000/')


In [27]:
import boto3

s3 = boto3.client(
    's3',
    endpoint_url='http://minio:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    region_name='us-east-1'
)

print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': '188763B2ACB44676', 'HostId': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'accept-ranges': 'bytes', 'content-length': '543', 'content-type': 'application/xml', 'server': 'MinIO', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'vary': 'Origin, Accept-Encoding', 'x-amz-id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'x-amz-request-id': '188763B2ACB44676', 'x-content-type-options': 'nosniff', 'x-ratelimit-limit': '2041', 'x-ratelimit-remaining': '2041', 'x-xss-protection': '1; mode=block', 'date': 'Sun, 04 Jan 2026 01:43:17 GMT'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'raw', 'CreationDate': datetime.datetime(2026, 1, 4, 1, 26, 24, 627000, tzinfo=tzlocal())}, {'Name': 'staging', 'CreationDate': datetime.datetime(2026, 1, 4, 1, 26, 24, 662000, tzinfo=tzlocal())}, {'Name': 'warehouse', 'CreationDate': datetime.datetime(2026, 1, 4, 1, 26, 24, 59

In [31]:
spark.sql("""SHOW TABLES IN polaris.olist_raw;""").show()

+---------+---------------+-----------+
|namespace|      tableName|isTemporary|
+---------+---------------+-----------+
|olist_raw|test_connection|      false|
+---------+---------------+-----------+



In [ ]:
spark.sql("""DROP TABLE polaris.olist_raw.test_connection;""")

In [6]:
spark.sql("""SHOW NAMESPACES IN polaris;""").show()

[02:20:02] [Thread-3] INFO  org.apache.iceberg.rest.auth.AuthManagers - Loading AuthManager implementation: org.apache.iceberg.rest.auth.OAuth2Manager
[02:20:04] [Thread-3] INFO  org.apache.iceberg.CatalogUtil - Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO
+---------+
|namespace|
+---------+
+---------+



In [30]:
spark.sql("""
DROP SCHEMA polaris.olist_raw CASCADE;
""")

Py4JJavaError: An error occurred while calling o43.sql.
: org.apache.iceberg.exceptions.NamespaceNotEmptyException: Namespace olist_raw is not empty
	at org.apache.iceberg.rest.ErrorHandlers$DropNamespaceErrorHandler.accept(ErrorHandlers.java:203)
	at org.apache.iceberg.rest.ErrorHandlers$DropNamespaceErrorHandler.accept(ErrorHandlers.java:197)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:240)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:336)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:297)
	at org.apache.iceberg.rest.BaseHTTPClient.delete(BaseHTTPClient.java:55)
	at org.apache.iceberg.rest.RESTSessionCatalog.dropNamespace(RESTSessionCatalog.java:640)
	at org.apache.iceberg.catalog.BaseSessionCatalog$AsCatalog.dropNamespace(BaseSessionCatalog.java:143)
	at org.apache.iceberg.rest.RESTCatalog.dropNamespace(RESTCatalog.java:247)
	at org.apache.iceberg.spark.SparkCatalog.dropNamespace(SparkCatalog.java:535)
	at org.apache.spark.s